In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  8 14:11:09 2020

@author: mor
"""

import networkx as nx
import cloudpickle as cp
from urllib.request import urlopen
import nxviz as nv
import matplotlib.pyplot as plt
from nxviz import CircosPlot
from nxviz import ArcPlot
from itertools import combinations


In [ ]:


#path='https://assets.datacamp.com/production/repositories/580/datasets/69ada08d5cce7f35f38ffefe8f2291b7cfcd6000/github_users.p'
#
#T = cp.load(urlopen(path))
#nx.write_gpickle(T, "data/github.p")



In [ ]:


G=nx.read_gpickle("data/github.p")



In [ ]:


# Import necessary modules
import matplotlib.pyplot as plt
import networkx as nx

# Plot the degree distribution of the GitHub collaboration network
plt.hist(list(nx.degree_centrality(G).values()))
plt.show()



In [ ]:


nodes_li=['u41', 'u69', 'u96', 'u156', 'u297', 'u298', 'u315', 'u322', 'u435', 'u440', 'u640', 'u655', 'u698',
 'u821', 'u863', 'u901', 'u914', 'u1254', 'u1407', 'u1468', 'u1908', 'u2022', 'u2066', 'u2137', 'u2289', 'u2482',
 'u2552', 'u2643', 'u2737', 'u2906', 'u3083', 'u3174', 'u3231', 'u3243', 'u3271', 'u3658', 'u3974', 'u3979',
 'u4159', 'u4199', 'u4329', 'u4412', 'u4513', 'u4710', 'u4761', 'u4953', 'u5082', 'u5337', 'u5693', 'u5993',
 'u6081', 'u7418', 'u7623', 'u7963', 'u8135', 'u9866', 'u9869', 'u9997', 'u10090', 'u10340', 'u10500', 'u10603',
 'u14964']

G_sub=G.subgraph(nodes_li)



In [ ]:


# Import necessary modules
import matplotlib.pyplot as plt
import networkx as nx

# Plot the degree distribution of the GitHub collaboration network
plt.hist(list(nx.betweenness_centrality(G_sub).values()))
plt.show()




In [ ]:


attrs = {n: {'grouping': 0} if n!='u14964' else {'grouping': 1} for n in G_sub.nodes()}
nx.set_node_attributes(G_sub, attrs)
 



In [ ]:


# Import necessary modules
from nxviz import MatrixPlot
import matplotlib.pyplot as plt

# Calculate the largest connected component subgraph: largest_ccs
largest_ccs = sorted(nx.connected_component_subgraphs(G_sub), key=lambda x: len(x))[-1]

# Create the customized MatrixPlot object: h
h = MatrixPlot(largest_ccs,node_grouping='grouping')

# Draw the MatrixPlot to the screen
h.draw()
plt.show()



In [ ]:


# Import necessary modules
from nxviz.plots import ArcPlot
import matplotlib.pyplot as plt

# Iterate over all the nodes in G, including the metadata
for n, d in G_sub.nodes(data=True):

    # Calculate the degree of each node: G.node[n]['degree']
    G_sub.node[n]['degree'] = nx.degree(G_sub,n)

# Create the ArcPlot object: a
a = ArcPlot(G_sub,node_order='degree')

# Draw the ArcPlot to the screen
a.draw()
plt.show()




In [ ]:



# Import necessary modules
from nxviz import CircosPlot
import matplotlib.pyplot as plt

# Iterate over all the nodes, including the metadata
for n, d in G_sub.nodes(data=True):

    # Calculate the degree of each node: G.node[n]['degree']
    G_sub.node[n]['degree'] = nx.degree(G_sub,n)

# Create the CircosPlot object: c
c = CircosPlot(G_sub,node_order='degree',node_grouping='grouping',node_color='grouping')

# Draw the CircosPlot object to the screen
c.draw()
plt.show()



In [ ]:


# Calculate the maximal cliques in G: cliques
cliques = nx.find_cliques(G_sub)

# Count and print the number of maximal cliques in G
print(len(list(cliques)))



In [ ]:


# Import necessary modules
import networkx as nx
from nxviz import CircosPlot
import matplotlib.pyplot as plt

# Find the author(s) that are part of the largest maximal clique: largest_clique
largest_clique = sorted(nx.find_cliques(G_sub), key=lambda x:len(x))[-1]

# Create the subgraph of the largest_clique: G_lc
G_lc = G_sub.subgraph(largest_clique)

# Create the CircosPlot object: c
c = CircosPlot(G_lc)

# Draw the CircosPlot to the screen
c.draw()
plt.show()




In [ ]:


# Compute the degree centralities of G: deg_cent
deg_cent = nx.degree_centrality(G)

# Compute the maximum degree centrality: max_dc
max_dc = max(deg_cent.values())

# Find the user(s) that have collaborated the most: prolific_collaborators
prolific_collaborators = [n for n, dc in deg_cent.items() if dc == max_dc]

# Print the most prolific collaborator(s)
print(prolific_collaborators)



In [ ]:

# Import necessary modules
from nxviz import ArcPlot
import matplotlib.pyplot as plt

# Identify the largest maximal clique: largest_max_clique
largest_max_clique = set(sorted(nx.find_cliques(G_sub), key=lambda x: len(x))[-1])

#We must explicitly ask for a .copy() of the graph to obtain a mutatable version.
# Create a subgraph from the largest_max_clique: G_lmc
G_lmc = G_sub.subgraph(largest_max_clique).copy()  

print(f'before: size of graph {len(G_lmc)}')

# Go out 1 degree of separation
for node in list(G_lmc.nodes()):
    G_lmc.add_nodes_from(G_sub.neighbors(node))
    G_lmc.add_edges_from(zip([node]*len(list(G_sub.neighbors(node))), G_sub.neighbors(node)))

# Record each node's degree centrality score
for n in G_lmc.nodes():
    G_lmc.node[n]['degree centrality'] = nx.degree_centrality(G_lmc)[n]

print(f'after: size of graph {len(G_lmc)}')

# Create the ArcPlot object: a
a = ArcPlot(G_lmc,node_order='degree centrality')

# Draw the ArcPlot to the screen
a.draw()
plt.show()




In [ ]:


# Import necessary modules
from itertools import combinations
from collections import defaultdict

# Initialize the defaultdict: recommended
recommended = defaultdict(int)

# Iterate over all the nodes in G
for n, d in G_sub.nodes(data=True):

    # Iterate over all possible triangle relationship combinations
    for n1, n2 in combinations(G_sub.neighbors(n), 2):

        # Check whether n1 and n2 do not have an edge
        if not G_sub.has_edge(n1, n2):

            # Increment recommended
            recommended[(n1, n2)] += 1

# Identify the top 10 pairs of users
all_counts = sorted(recommended.values())
top10_pairs = [pair for pair, count in recommended.items() if count > all_counts[-10]]
print(top10_pairs)

